In [ ]:
import geopandas as gpd
from tqdm import tqdm
from shapely.geometry import Point, Polygon, MultiPolygon
import shapely

In [ ]:
def queen_corners(gdf, sensitivity=2):
    """
    Experimental: Fix unprecise corners.
    """
    tessellation = gdf.copy()
    changes = {}
    qid = 0
    sindex = tessellation.sindex

    for ix, row in tqdm(tessellation.iterrows(), total=tessellation.shape[0]):
        corners = []
        change = []

        cell = row.geometry
        coords = cell.exterior.coords
        for i in coords:
            point = Point(i)
            possible_matches_index = list(sindex.intersection(point.bounds))
            possible_matches = tessellation.iloc[possible_matches_index]
            precise_matches = sum(possible_matches.intersects(point))
            if precise_matches > 2:
                corners.append(point)

        if len(corners) > 2:
            for c, it in enumerate(corners):
                next_c = c + 1
                if c == (len(corners) - 1):
                    next_c = 0
                if corners[c].distance(corners[next_c]) < sensitivity:
                    change.append([corners[c], corners[next_c]])
        elif len(corners) == 2:
            if corners[0].distance(corners[1]) > 0:
                if corners[0].distance(corners[1]) < sensitivity:
                    change.append([corners[0], corners[1]])

        if change:
            for points in change:
                x_new = np.mean([points[0].x, points[1].x])
                y_new = np.mean([points[0].y, points[1].y])
                new = [(x_new, y_new), id]
                changes[(points[0].x, points[0].y)] = new
                changes[(points[1].x, points[1].y)] = new
                qid = qid + 1

    for ix, row in tqdm(tessellation.iterrows(), total=tessellation.shape[0]):
        cell = row.geometry
        coords = list(cell.exterior.coords)

        moves = {}
        for x in coords:
            if x in changes.keys():
                moves[coords.index(x)] = changes[x]
        keys = list(moves.keys())
        delete_points = []
        for move, k in enumerate(keys):
            if move < len(keys) - 1:
                if (
                    moves[keys[move]][1] == moves[keys[move + 1]][1]
                    and keys[move + 1] - keys[move] < 5
                ):
                    delete_points = delete_points + (
                        coords[keys[move] : keys[move + 1]]
                    )
                    # change the code above to have if based on distance not number

        newcoords = [changes[x][0] if x in changes.keys() else x for x in coords]
        for coord in newcoords:
            if coord in delete_points:
                newcoords.remove(coord)
        if coords != newcoords:
            if not cell.interiors:
                # newgeom = Polygon(newcoords).buffer(0)
                be = Polygon(newcoords).exterior
                mls = be.intersection(be)
                if len(list(shapely.ops.polygonize(mls))) > 1:
                    newgeom = MultiPolygon(shapely.ops.polygonize(mls))
                    geoms = []
                    for g, n in enumerate(newgeom):
                        geoms.append(newgeom[g].area)
                    newgeom = newgeom[geoms.index(max(geoms))]
                else:
                    newgeom = list(shapely.ops.polygonize(mls))[0]
            else:
                newgeom = Polygon(newcoords, holes=cell.interiors)
            tessellation.loc[ix, "geometry"] = newgeom
    return tessellation

In [ ]:
path = 'folder/AMS'
tess = gpd.read_file(path + '/elements.gpkg', layer='tessellation')

In [ ]:
queen = queen_corners(tess)

In [ ]:
queen.to_file(path + 'queen.gpkg', layer='tessellation', driver='GPKG')